# Test

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline

import csv

#!pip install request
import requests # library to handle requests

#!pip install beautifulsoup4 
from bs4 import BeautifulSoup

#!pip install lxml

#!pip install html5lib

#!conda install -c conda-forge geopy --yes # module Niminatim
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  52.77 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.86 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.92 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.26 MB/s


# Coursera Course - Applied Data Science Capstone
## Housing prices - Hamburg

### 1. Introduction

#### 1.1 Description of the problem


#### 1.2 Description of the data


##### i) Boroughs and neighborhoods

##### ii) Post codes

##### iv) Venues for each Post code?, Neighborhood? or Boroughs

##### v) Transport, Train stations, S-Bahn stations, Subway stations, Bus stations

##### vi) Housing prices, archiv and current 

####  Scrap wikipedia - Boroughs and Neighborhoods of Hamburg

In [3]:
wiki_borough_link = "https://de.wikipedia.org/wiki/Liste_der_Bezirke_und_Stadtteile_Hamburgs#Stadtteile"

In [4]:
r= requests.get(wiki_borough_link)
soup = BeautifulSoup(r.text,'lxml')

In [5]:
#print(soup.prettify())

In [6]:
# define columns
col_names = ['neighborhood','borough','area','population', 'coord_nord','coord_east'] 

# initiate dataframe
df_neighborhoods = pd.DataFrame(columns=col_names)

In [7]:
# Function to transform coordinates from geographic to decimal
def convert_decimal(coordinate):
    coord = ''.join(coordinate.split())[:-2].upper()
    coord_d, coord_m, coord_s = coord.replace("°","_").replace("′","_").split('_')                            
    coord = "{0:.5f}".format(float(coord_d) + (float(coord_m) / 60) + (float(coord_s) / 3600))
    return coord

In [8]:
table = soup.find_all('table', class_='wikitable sortable')
# iterate over it
wikidatasaved = []
for item in table[1].find('tbody').find_all('tr'):
    wikidata = []
    for data in item.find_all('td'):
        wikidata.append(data.text.strip())
    
    #print(wikidata[0:8])
    if len(wikidata) > 0:
            wikidata[0], b = wikidata[0].split("!")
            b, wikidata[3] = wikidata[3].split("!")
            coord_n, coord_e = wikidata[6].split(",")
            coord_n = convert_decimal(coord_n)
            coord_e = convert_decimal(coord_e)
            df_neighborhoods = df_neighborhoods.append({'neighborhood': wikidata[0],'borough': wikidata[2], 'area' : wikidata[3], 'population' : wikidata[4], 'coord_nord':coord_n, 'coord_east':coord_e}, ignore_index=True)
            

In [9]:
df_neighborhoods.head()

,neighborhood,borough,area,population,coord_nord,coord_east
0,Hamburg-Altstadt,Hamburg-Mitte,"2,4",2305,53.55000,10.00000
1,HafenCity,Hamburg-Mitte,"2,2",3627,53.54111,10.00028
2,Neustadt,Hamburg-Mitte,"2,3",12.719,53.55194,9.98556
3,St.Pauli,Hamburg-Mitte,"2,5",22.501,53.55694,9.96389
4,St.Georg,Hamburg-Mitte,"2,4",11.055,53.55500,10.01222


####  Scrap hamburg.de - post codes of hamburg

In [10]:
plz_link = "https://www.hamburg.de/postleitzahlen/"

In [11]:
r= requests.get(plz_link)
soup = BeautifulSoup(r.text,'lxml')
#print(soup.prettify())

In [12]:
# define columns
col_plz_names = ['neighborhood','post codes'] 

# initiate dataframe
df_plz = pd.DataFrame(columns=col_plz_names)

In [13]:
table = soup.find_all('table')
# iterate over it
datasaved = []
for item in table[0].find('tbody').find_all('tr'):
    plzdata = []
    for data in item.find_all('td'):
        if data.text.strip() != '':
            plzdata.append(data.text.strip())
    df_plz = df_plz.append({'neighborhood': plzdata[0],'post codes': ",".join(plzdata[1:len(plzdata)])}, ignore_index=True)

df_plz.head()

,neighborhood,post codes
0,Allermöhe,"21035,21037,21113"
1,Alsterdorf,"20251,22297,22335,22337"
2,Altengamme,"21029,21039"
3,Altenwerder,21129
4,Altona-Altstadt,"20359,22765,22767,22769"


In [49]:
df_neigh = pd.merge(df_neighborhoods, df_plz, on='neighborhood',how='left')
df_neigh.sort_values(by='neighborhood').head()

,neighborhood,borough,area,population,coord_nord,coord_east,post codes
81,Allermohe,Bergedorf,"8,6",1361,53.48611,10.15417,NaN
45,Alsterdorf,Hamburg-Nord,"3,2",15.031,53.61083,10.01306,"20251,22297,22335,22337"
76,Altengamme,Bergedorf,"15,6",2246,53.42472,10.26250,"21029,21039"
98,Altenwerder,Harburg,"6,7",3,53.50694,9.91778,21129
19,Altona-Altstadt,Altona,"2,8",22.946,53.54889,9.94778,"20359,22765,22767,22769"


In [50]:
import math
from difflib import SequenceMatcher

In [54]:
# fucntion to evaluate the similarity of two strings
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

for index, row in df_neigh.iterrows():
     if isinstance(row[6],str) != True:
            print(row[0],row[6])
            for index_plz, row_plz in df_plz.iterrows():
                
                if similar(row[0],row_plz[0]) > 0.8:
                    print(row[0],row[6],"<-",row_plz[0]," ",row_plz[1])
                    #row[6] = row_plz[1]

HafenCity nan
HafenCity nan <- Hafencity   20457,20539
St.Pauli nan
St.Pauli nan <- St. Pauli   20354,20355,20357,20359,22767,22769
St.Georg nan
St.Georg nan <- St. Georg   20095,20097,20099
Sulldorf nan
Eimsbuttel nan
Eimsbuttel nan <- Eimsbüttel   20144,20253,20255,20257,20259,20357
Barmbek-Sud nan
Barmbek-Sud nan <- Barmbek-Süd   22081,22083,22085,22305
Fuhlsbuttel nan
Fuhlsbuttel nan <- Fuhlsbüttel   22335,22339,22415
Wellingsbuttel nan
Wellingsbuttel nan <- Wellingsbüttel   22391,22393
Poppenbuttel nan
Poppenbuttel nan <- Poppenbüttel   22391,22393,22395,22399
Hummelsbuttel nan
Hummelsbuttel nan <- Hummelsbüttel   22339,22391,22399,22145,22417
Lohbrugge nan
Lohbrugge nan <- Lohbrügge   21031,21033,22113,22115
Allermohe nan
Allermohe nan <- Allermöhe   21035,21037,21113
Neuallermohe nan
Neuallermohe nan <- Neuallermöhe   21035
Ronneburg nan
Ronneburg nan <- Rönneburg   21077,21079
Eissendorf nan
Eissendorf nan <- Eißendorf   21073,21075,21077


In [48]:
#df_neigh.sort_values(by='neighborhood').head()
df_neigh[df_neigh.neighborhood.str.startswith('S')]
#df_neighborhoods[df_neighborhoods.neighborhood.str.startswith('S')]
#df_plz[df_plz.neighborhood.str.startswith('S')]


,neighborhood,borough,area,population,coord_nord,coord_east,post codes
3,St.Pauli,Hamburg-Mitte,"2,5",22.501,53.55694,9.96389,NaN
4,St.Georg,Hamburg-Mitte,"2,4",11.055,53.55500,10.01222,NaN
15,Steinwerder,Hamburg-Mitte,"7,4",38,53.53444,9.95722,"20457,21107"
20,Sternschanze,Altona,"0,5",8080,53.56167,9.96222,"20537,20539,22767,22769"
31,Sulldorf,Altona,"5,7",9542,53.58500,9.79417,NaN
39,Schnelsen,Eimsbüttel,"9,0",29.578,53.63333,9.91667,"22457,22459"
41,Stellingen,Eimsbüttel,"5,8",25.753,53.59222,9.92861,"20255,22525,22527,22529,22769"
62,Steilshoop,Wandsbek,"2,5",19.662,53.61028,10.05917,"22177,22309"
64,Sasel,Wandsbek,"8,4",23.647,53.65389,10.11194,"22159,22391,22393,22395"
85,Spadenland,Bergedorf,"3,4",514,53.50000,10.23333,21037
